# Path

In [61]:
import sys
print(sys.argv[0])
import os
print (os.getcwd())#获得当前工作目录
print (os.path.abspath('.'))#获得当前工作目录
print (os.path.abspath('..'))#获得当前工作目录的父目录
print (os.path.abspath(os.curdir))#获得当前工作目录
# print(os.path.abspath('..')+'/src/sample_code_submission')
# os.chdir(os.path.abspath('..')+'/src/sample_code_submission')
# os.path.abspath('..')+'/src/sample_code_submission'
from pprint import pprint

/Users/xijunli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py
/Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission
/Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission
/Users/xijunli/Desktop/KDDCup2019/src
/Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission


# Data Input

In [11]:
# pylint: disable=wrong-import-order, wrong-import-position, import-error
# pylint: disable=missing-docstring
import base64
from datetime import datetime
import os
from os.path import join
import sys

#  os.system("pip3 install cryptography")


def mprint(msg):
    """info"""
    cur_time = datetime.now().strftime('%m-%d %H:%M:%S')
    print(f"INFO  [{cur_time}] {msg}")


mprint("Import Model")

import json
import signal
import time
from contextlib import contextmanager
import numpy as np
import pandas as pd
import math

TYPE_MAP = {
    'time': str,
    'cat': str,
    'multi-cat': str,
    'num': np.float64
}


class TimeoutException(Exception):
    pass


class Timer:
    def __init__(self):
        self.duration = 0
        self.total = None
        self.remain = None
        self.exec = None

    def set(self, time_budget):
        self.total = time_budget
        self.remain = time_budget
        self.exec = 0

    @contextmanager
    def time_limit(self, pname):
        def signal_handler(signum, frame):
            raise TimeoutException("Timed out!")
        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(self.remain)
        start_time = time.time()
        try:
            yield
        finally:
            exec_time = time.time() - start_time
            signal.alarm(0)
            self.exec += exec_time
            self.duration += exec_time
            remain_time = math.ceil(self.total - self.exec)
            self.remain = remain_time

            mprint(f'{pname} success, time spent so far {self.exec} sec')


def read_train(datapath, info):
    train_data = {}
    for table_name, columns in info['tables'].items():
        mprint(f'Table name: {table_name}')

        table_dtype = {key: TYPE_MAP[val] for key, val in columns.items()}

        if table_name == 'main':
            table_path = join(datapath, 'train', 'main_train.data')
        else:
            table_path = join(datapath, 'train', f'{table_name}.data')

        date_list = [key for key, val in columns.items() if val == 'time']

        train_data[table_name] = pd.read_csv(
            table_path, sep='\t', dtype=table_dtype, parse_dates=date_list,
            date_parser=lambda millisecs: millisecs if np.isnan(
                float(millisecs)) else datetime.fromtimestamp(
                    float(millisecs)/1000))

    # get train label
    train_label = pd.read_csv(
        join(datapath, 'train', 'main_train.solution'))['label']
    return train_data, train_label


def read_info(datapath):
    mprint('Read info')
    with open(join(datapath, 'train', 'info.json'), 'r') as info_fp:
        info = json.load(info_fp)
    mprint(f'Time budget for this task is {info["time_budget"]} sec')
    return info


def read_test(datapath, info):
    # get test data
    main_columns = info['tables']['main']
    table_dtype = {key: TYPE_MAP[val] for key, val in main_columns.items()}

    table_path = join(datapath, 'test', 'main_test.data')

    date_list = [key for key, val in main_columns.items() if val == 'time']

    test_data = pd.read_csv(
        table_path, sep='\t', dtype=table_dtype, parse_dates=date_list,
        date_parser=lambda millisecs: millisecs if np.isnan(
            float(millisecs)) else datetime.fromtimestamp(
                float(millisecs) / 1000))
    return test_data


def write_predict(output_dir, dataname, prediction):
    os.makedirs(output_dir, exist_ok=True)
    prediction.rename('label', inplace=True)
    prediction.to_csv(
        join(output_dir, f'{dataname}.predict'), index=False, header=True)


INFO  [04-21 17:01:19] Import Model


In [56]:
ROOT_DIR = os.path.abspath('../../starting_kit_0401/')
DIRS = {
        'input': join(ROOT_DIR, 'sample_data'),
        'output': join(ROOT_DIR, 'sample_predictions'),
        'program': join(ROOT_DIR, 'ingestion_program'),
        'submission': join(ROOT_DIR, 'sample_code_submission')
}

datanames = sorted(os.listdir(DIRS['input']))
mprint(f'Datanames: {datanames}')
timer = Timer()
dataname = datanames[0]
predictions = {}
mprint(f'Read data: {dataname}')
datapath = join(DIRS['input'], dataname)
info = read_info(datapath)
timer.set(info['time_budget'])
train_data, train_label = read_train(datapath, info)

INFO  [04-21 22:39:11] Datanames: ['K', 'L']
INFO  [04-21 22:39:11] Read data: K
INFO  [04-21 22:39:11] Read info
INFO  [04-21 22:39:11] Time budget for this task is 6000 sec
INFO  [04-21 22:39:11] Table name: main
INFO  [04-21 22:39:11] Table name: table_1
INFO  [04-21 22:39:11] Table name: table_2
INFO  [04-21 22:39:11] Table name: table_3


# AutoML Model

## Construct Model

In [91]:
import os
import time
from collections import defaultdict, deque

import numpy as np
import pandas as pd

import CONSTANT
from util import Config, Timer, log, timeit

NUM_OP = [np.std, np.mean]

config = Config(info)
tables = copy.deepcopy(train_data)

## fit

### clean_tables

In [74]:
for tname in tables:
    log(f"cleaning table {tname}")
    clean_df(tables[tname])

cleaning table main

Start [clean_df]:
----Start [fillna]:
----End   [fillna]. Time elapsed: 0.00 sec.
End   [clean_df]. Time elapsed: 0.00 sec.
cleaning table table_1

Start [clean_df]:
----Start [fillna]:
----End   [fillna]. Time elapsed: 0.02 sec.
End   [clean_df]. Time elapsed: 0.02 sec.
cleaning table table_2

Start [clean_df]:
----Start [fillna]:
----End   [fillna]. Time elapsed: 0.01 sec.
End   [clean_df]. Time elapsed: 0.01 sec.
cleaning table table_3

Start [clean_df]:
----Start [fillna]:
----End   [fillna]. Time elapsed: 0.00 sec.
End   [clean_df]. Time elapsed: 0.00 sec.


### X = merge_table(Xs, self.config)

In [76]:
from collections import defaultdict, deque

import numpy as np
import pandas as pd

import CONSTANT
from util import Config, Timer, log, timeit

# def merge_table(tables, config):

graph = defaultdict(list)
for rel in config['relations']:
    ta = rel['table_A']
    tb = rel['table_B']
    graph[ta].append({
            "to": tb,
            "key": rel['key'],
            "type": rel['type']
    })
    graph[tb].append({
            "to": ta,
            "key": rel['key'],
            "type": '_'.join(rel['type'].split('_')[::-1])
    })


#### bfs(CONSTANT.MAIN_TABLE_NAME, graph, config['tables'])

In [112]:
tconfig = config["tables"]
root_name = CONSTANT.MAIN_TABLE_NAME

tconfig[CONSTANT.MAIN_TABLE_NAME]['depth'] = 0
queue = deque([root_name])
while queue:
    u_name = queue.popleft()
    for edge in graph[u_name]:
        v_name = edge['to']
        if 'depth' not in tconfig[v_name]:
            tconfig[v_name]['depth'] = tconfig[u_name]['depth'] + 1
            queue.append(v_name)

pprint(graph)
pprint(config)

import pdb
dfs(CONSTANT.MAIN_TABLE_NAME, config, tables, graph)

defaultdict(<class 'list'>,
            {'main': [{'key': ['c_01'], 'to': 'table_1', 'type': 'many_to_one'},
                      {'key': ['c_02'], 'to': 'table_2', 'type': 'many_to_one'},
                      {'key': ['c_02'],
                       'to': 'table_3',
                       'type': 'many_to_one'}],
             'table_1': [{'key': ['c_01'],
                          'to': 'main',
                          'type': 'one_to_many'}],
             'table_2': [{'key': ['c_02'],
                          'to': 'main',
                          'type': 'one_to_many'}],
             'table_3': [{'key': ['c_02'],
                          'to': 'main',
                          'type': 'one_to_many'}]})
{'start_time': 1550654179, 'time_budget': 6000, 'time_col': 't_01', 'tables': {'main': {'type': {'n_1': 'num', 'n_2': 'num', 'n_3': 'num', 't_01': 'time', 'c_02': 'cat', 'c_01': 'cat'}, 'depth': 0}, 'table_1': {'type': {'c_01': 'cat', 'c_1': 'cat', 'c_2': 'cat', 'n_1': 'num', 'n

BdbQuit: 

#### dfs(CONSTANT.MAIN_TABLE_NAME, config, tables, graph)

In [108]:
def dfs(u_name, config, tables, graph):
    pdb.set_trace()
    u = tables[u_name]
    log(f"enter {u_name}")
    for edge in graph[u_name]:
        v_name = edge['to']
        if config['tables'][v_name]['depth'] <= config['tables'][u_name]['depth']:
            continue

        v = dfs(v_name, config, tables, graph)
        key = edge['key']
        type_ = edge['type']

        if config['time_col'] not in u and config['time_col'] in v:
            continue

        if config['time_col'] in u and config['time_col'] in v:
            log(f"join {u_name} <--{type_}--t {v_name}")
            u = temporal_join(u, v, v_name, key, config['time_col'])
        else:
            log(f"join {u_name} <--{type_}--nt {v_name}")
            u = join(u, v, v_name, key, type_)

        del v

    log(f"leave {u_name}")
    return u